# 데이터 검색

이 노트북에서는 Azure AI Search의 Knowledge Base 를 이용한 호출을 수행합니다.

## 0. 필요한 라이브러리 설치

In [ ]:
##############################################
# 0. 라이브러리 설치 (최초 1회 실행)
##############################################
%pip install --quiet --upgrade pip
%pip install --quiet python-dotenv requests azure-identity azure-search-documents openai
%pip install --quiet azure-search-documents --pre --upgrade

## 1. 환경 변수 설정

In [ ]:
import os
from dotenv import load_dotenv

##############################################
# 1. 환경 변수 로드 및 확인
##############################################
load_dotenv()

# Azure AI Search
AZURE_SEARCH_SERVICE_ENDPOINT = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")

# 환경 변수 로드
AZURE_KNOWLEDGE_BASE_NAME = os.getenv("AZURE_KNOWLEDGE_BASE_NAME")
AZURE_KNOWLEDGE_SOURCE_NAME = os.getenv("AZURE_KNOWLEDGE_SOURCE_NAME")


# print(f"Azure AISearch Endpoint: {AZURE_SEARCH_SERVICE_ENDPOINT}")
# print(f"Azure AISearch KNOWLEDGE_BASE_NAME: {AZURE_KNOWLEDGE_BASE_NAME}")
# print(f"Azure AISearch KNOWLEDGE_SOURCE_NAME: {AZURE_KNOWLEDGE_SOURCE_NAME}")  

## 2. Knowledge Base 호출

In [ ]:
import json
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

from azure.search.documents.knowledgebases import KnowledgeBaseRetrievalClient
from azure.search.documents.knowledgebases.models import (
    KnowledgeBaseMessage, 
    KnowledgeBaseMessageTextContent, 
    KnowledgeBaseRetrievalRequest, 
    SearchIndexKnowledgeSourceParams,
)

##############################################
# 2-1. Azure 인증 및 클라이언트 생성
##############################################
credential = DefaultAzureCredential()

##############################################
# 2-2. Knowledge Base 검색 수행
##############################################

# Knowledge Base 클라이언트 생성
kb_client = KnowledgeBaseRetrievalClient(
    endpoint=AZURE_SEARCH_SERVICE_ENDPOINT,
    knowledge_base_name=AZURE_KNOWLEDGE_BASE_NAME,
    credential=credential,
)

# 검색 쿼리
search_query = "AI 기반 정밀 온도 제어 시스템에 대해서 알려줘"

# Knowledge Base 검색 요청 생성
request = KnowledgeBaseRetrievalRequest(
    messages=[
        KnowledgeBaseMessage(
            role="user",
            content=[KnowledgeBaseMessageTextContent(text=search_query)]
        ),
    ],
    knowledge_source_params=[
        SearchIndexKnowledgeSourceParams(
            knowledge_source_name=AZURE_KNOWLEDGE_SOURCE_NAME,
            include_references=True,
            include_reference_source_data=True,
            always_query_source=True,
        )
    ],
    include_activity=True,
)

print(f"검색어: '{search_query}'")
print(f"\n{'='*60}")
print("[Knowledge Base 검색 결과]")
print(f"{'='*60}\n")

result = kb_client.retrieve(request)
print(result.response[0].content[0].text)

print(f"\n{'='*60}")

## 3. Knowledge Base 상세 결과 출력

In [ ]:
##############################################
# 3-1. Activity 및 References 출력
##############################################

# Display the response, activity, and references
response_contents = []
activity_contents = []
references_contents = []

# Print the three string values
if result.activity:
    activity_content = json.dumps([a.as_dict() for a in result.activity], indent=2)
else:
    activity_content = "No activity found on 'result'"
    
activity_contents.append(activity_content)

print(f"\n{'='*60}")
print("Activity_content")
print(f"{'='*60}\n")
print(activity_content)


if result.references:
    references_content = json.dumps([r.as_dict() for r in result.references], indent=2)
else:
    references_content = "No references found on 'result'"
    
references_contents.append(references_content)

print(f"\n{'='*60}")
print("References_content")
print(f"{'='*60}\n")
print(references_content)